In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromiumService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup as soup

import time
import pandas as pd
from datetime import datetime

In [2]:
options = [
        "--headless",
        "--window-size=1920,1200",
        "--start-maximized",
        "--no-sandbox",
        "--disable-dev-shm-usage",
        "--disable-gpu",
        "--ignore-certificate-errors",
        "--disable-extensions",
        "--disable-popup-blocking",
        "--disable-notifications",
        "--remote-debugging-port=9222", #https://stackoverflow.com/questions/56637973/how-to-fix-selenium-devtoolsactiveport-file-doesnt-exist-exception-in-python
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
        "--disable-blink-features=AutomationControlled",
        ]

In [3]:
service = ChromiumService(executable_path=r"C:\Users\quant\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
        
chrome_options = Options() 
for option in options:
    chrome_options.add_argument(option)

driver = webdriver.Chrome(service=service, options=chrome_options)

In [4]:
def parse_ids(data_table):
    CLASS_ID = 'Anchor_anchor__cSc3P' #determined by visual inspection of page source code

    # get all the links
    links = data_table.find_all('a', {'class':CLASS_ID})

    links_list = [i.get("href") for i in links]

    # create a series using last 10 digits of the appropriate links
    team_id = pd.Series([i[-10:] for i in links_list if ('stats' in i)])
    game_id = pd.Series([i[-10:] for i in links_list if ('/game/' in i)])
    
    return team_id, game_id

def scrape_to_dataframe(driver, season, dateFrom, dateTo, seasonType):
    url = "https://www.nba.com/stats/teams/boxscores?SeasonType=" + seasonType
    if not season:
        url = url + "&DateFrom=" + dateFrom + "&DateTo=" + dateTo
    else:
        if dateFrom == "NONE" and dateTo == "NONE":
            url = url + "&Season=" + season
        else:
            url = url + "&Season=" + season + "&DateFrom=" + dateFrom + "&DateTo=" + dateTo

    print(f"Scraping {url}")

    driver.get(url)
    time.sleep(10)
    source = soup(driver.page_source, 'html.parser')

    CLASS_ID_TABLE = 'Crom_table__p1iZz' #determined by visual inspection of page source code
    data_table = source.find('table', {'class':CLASS_ID_TABLE})
    if data_table is None:
        return pd.DataFrame()
    
    CLASS_ID_PAGINATION = "Pagination_pageDropdown__KgjBU" #determined by visual inspection of page source code
    pagination = source.find('div', {'class':CLASS_ID_PAGINATION})
    if pagination is not None:
        # if multiple pages, first activate pulldown option for All pages to show all rows on one page
        CLASS_ID_DROPDOWN = "DropDown_select__4pIg9" #determined by visual inspection of page source code
        page_dropdown = driver.find_element(By.XPATH, "//*[@class='" + CLASS_ID_PAGINATION + "']//*[@class='" + CLASS_ID_DROPDOWN + "']")
    
        page_dropdown.send_keys("ALL") # show all pages
        #page_dropdown.click() doesn't work in headless mode
        time.sleep(3)
        driver.execute_script('arguments[0].click()', page_dropdown) #click() didn't work in headless mode, used this workaround (https://stackoverflow.com/questions/57741875)
        
        #refresh page data now that it contains all rows of the table
        time.sleep(3)
        source = soup(driver.page_source, 'html.parser')
        data_table = source.find('table', {'class':CLASS_ID_TABLE})

    dfs = pd.read_html(str(data_table), header=0) 
    df = pd.concat(dfs)

    # pull out teams ids and game ids from hrefs and add these to the dataframe
    TEAM_ID, GAME_ID = parse_ids(data_table)
    df['TEAM_ID'] = TEAM_ID
    df['GAME_ID'] = GAME_ID

    return df

In [5]:
def convert_columns(df):
    drop_columns = ['MIN']
    df = df.drop(columns=drop_columns)

    mapper = {
         'Match Up': 'HOME',
         'Game Date': 'GAME_DATE_EST', 
         'W/L': 'HOME_TEAM_WINS',
         'FG%': 'FG_PCT',
         '3P%': 'FG3_PCT',
         'FT%': 'FT_PCT',
    }
    df = df.rename(columns=mapper)
    # make HOME true if @ is NOT in the text
    # each game has two rows, one for each team
    # Home team is always the team without the @
    # TEAM   MATCH UP
    # DAL    DAL @ POR  
    # POR    POR vs DAL 
    df['HOME'] = df['HOME'].apply(lambda x: 0 if '@' in x else 1)
    
    # convert wins to home team wins
    # incomplete games will be NaN
    df = df[df['HOME_TEAM_WINS'].notna()]
    # convert W/L to 1/0
    df['HOME_TEAM_WINS'] = df['HOME_TEAM_WINS'].apply(lambda x: 1 if 'W' in x else 0)
    # no need to do anything else, win/loss of visitor teams is not used in final dataframe
    
    #convert date format
    df['GAME_DATE_EST'] = pd.to_datetime(df['GAME_DATE_EST'])
    df['GAME_DATE_EST'] = df['GAME_DATE_EST'].dt.strftime('%Y-%m-%d')
    df['GAME_DATE_EST'] = pd.to_datetime(df['GAME_DATE_EST'])

    return df

In [6]:
def combine_home_visitor(df):
    # separate home vs visitor
    home_df = df[df['HOME'] == 1]
    visitor_df = df[df['HOME'] == 0]
    
    # HOME column no longer needed
    home_df = home_df.drop(columns='HOME')
    visitor_df = visitor_df.drop(columns='HOME')
    
    # HOME_TEAM_WINS and GAME_DATE_EST columns not needed for visitor
    visitor_df = visitor_df.drop(columns=['HOME_TEAM_WINS','GAME_DATE_EST'])
    
    # rename TEAM_ID columns
    home_df = home_df.rename(columns={'TEAM_ID':'HOME_TEAM_ID'})
    visitor_df = visitor_df.rename(columns={'TEAM_ID':'VISITOR_TEAM_ID'})
    
    # merge the home and visitor data
    df = pd.merge(home_df, visitor_df, how="left", on=["GAME_ID"],suffixes=('_home', '_away'))
    
    # add a column for SEASON
    # determine SEASON by parsing GAME_ID 
    # (e.g. 0022200192 1st 2 digits not used, 3rd digit 2 = regular season, 4th and 5th digit = SEASON)
    game_id = df['GAME_ID'].iloc[0]
    season = game_id[3:5]
    season = str(20) + season
    df['SEASON'] = season

    return df

In [7]:
def get_games(driver, season, dateFrom, dateTo):
    season_types = ["Regular+Season", "PlayIn", "Playoffs"]
    all_season_types = pd.DataFrame()

    for season_type in season_types:
        df = scrape_to_dataframe(driver, season, dateFrom, dateTo, season_type)
        if not(df.empty):
            df = convert_columns(df)
            df = combine_home_visitor(df)
            all_season_types = pd.concat([all_season_types, df], axis=0)

    return all_season_types

In [8]:
last_season = 2003
start_date = '2003-08-01'
new_games = pd.DataFrame()
df_season = pd.DataFrame()

for season in range(last_season, 2024):
    end_date = datetime.strptime(f"{season+1}-08-01", "%Y-%m-%d") # use August 1st to get all games from the current season
    print(f"Scraping season {season} from {start_date} to {end_date}")
    df_season = get_games(driver, str(season), str(start_date), str(end_date))
    new_games = pd.concat([new_games, df_season], axis=0)
    start_date = datetime.strptime(f"{season+1}-10-01", "%Y-%m-%d") # if more than 1 season, reset start date to beginning of next season

Scraping season 2003 from 2003-08-01 to 2004-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Regular+Season&Season=2003&DateFrom=2003-08-01&DateTo=2004-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=PlayIn&Season=2003&DateFrom=2003-08-01&DateTo=2004-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Playoffs&Season=2003&DateFrom=2003-08-01&DateTo=2004-08-01 00:00:00
Scraping season 2004 from 2004-10-01 00:00:00 to 2005-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Regular+Season&Season=2004&DateFrom=2004-10-01 00:00:00&DateTo=2005-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=PlayIn&Season=2004&DateFrom=2004-10-01 00:00:00&DateTo=2005-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Playoffs&Season=2004&DateFrom=2004-10-01 00:00:00&DateTo=2005-08-01 00:00:00
Scraping season 2005 from 2005-10-01 00:00:00 to 2006-08

In [9]:
new_games.to_csv('games.csv', index=False)
del [new_games]

In [10]:
games = pd.read_csv("games.csv")

In [11]:
games.columns

Index(['Team_home', 'GAME_DATE_EST', 'HOME_TEAM_WINS', 'PTS_home', 'FGM_home',
       'FGA_home', 'FG_PCT_home', '3PM_home', '3PA_home', 'FG3_PCT_home',
       'FTM_home', 'FTA_home', 'FT_PCT_home', 'OREB_home', 'DREB_home',
       'REB_home', 'AST_home', 'STL_home', 'BLK_home', 'TOV_home', 'PF_home',
       '+/-_home', 'HOME_TEAM_ID', 'GAME_ID', 'Team_away', 'PTS_away',
       'FGM_away', 'FGA_away', 'FG_PCT_away', '3PM_away', '3PA_away',
       'FG3_PCT_away', 'FTM_away', 'FTA_away', 'FT_PCT_away', 'OREB_away',
       'DREB_away', 'REB_away', 'AST_away', 'STL_away', 'BLK_away', 'TOV_away',
       'PF_away', '+/-_away', 'VISITOR_TEAM_ID', 'SEASON'],
      dtype='object')

In [12]:
#remove preseason games (GAME_ID begins with a 1)
games = games[games['GAME_ID'] > 20000000]

#flag postseason games (GAME_ID begins with >2)
games['PLAYOFF'] = (games['GAME_ID'] >= 30000000).astype('int8')

#remove duplicates (each GAME_ID should be unique)
games = games[~games.duplicated(subset=['GAME_ID'])]

#drop unnecessary fields
# drop_fields = ['Unnamed: 0']
# games = games.drop(drop_fields,axis=1)

games['TARGET'] = games['HOME_TEAM_WINS']

In [13]:
games.to_csv('transformed.csv', index=False)